In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!unzip -q '/content/drive/MyDrive/Диплом/DiabRetinaTrain.zip' -d '/content'

Mounted at /content/drive


In [ ]:
%pip install autokeras

In [ ]:
import tensorflow as tf
import os
from PIL import Image,ImageOps
import numpy as np
import random
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
TRAIN_PATH = '/content/Train'       # Папка для обучающего набора данных
TEST_PATH = '/content/Test'  # Папка для тестового набора данных
IMAGE_PATH = '/content/Train/'
TEST_SPLIT =0.1

In [ ]:
# Определение списка имен классов
CLASS_LIST = sorted(os.listdir(TRAIN_PATH))
# Определение количества классов
CLASS_COUNT = len(CLASS_LIST)
# Проверка результата
print(f'Количество классов: {CLASS_COUNT}, метки классов: {CLASS_LIST}')

Количество классов: 5, метки классов: ['0', '1', '2', '3', '4']


In [ ]:
data_files = []                           # Cписок путей к файлам картинок
data_labels = []                          # Список меток классов, соответствующих файлам

for class_label in range(CLASS_COUNT):    # Для всех классов по порядку номеров (их меток)
    class_name = CLASS_LIST[class_label]  # Выборка имени класса из списка имен
    class_path = IMAGE_PATH + class_name  # Формирование полного пути к папке с изображениями класса
    class_files = os.listdir(class_path)  # Получение списка имен файлов с изображениями текущего класса
    print(f'Размер класса {class_name} составляет {len(class_files)} фотографий')

    # Добавление к общему списку всех файлов класса с добавлением родительского пути
    data_files += [f'{class_path}/{file_name}' for file_name in class_files]

    # Добавление к общему списку меток текущего класса - их ровно столько, сколько файлов в классе
    data_labels += [class_label] * len(class_files)

print('Общий размер базы для обучения:', len(data_labels))

Размер класса 0 составляет 1615 фотографий
Размер класса 1 составляет 1513 фотографий
Размер класса 2 составляет 1491 фотографий
Размер класса 3 составляет 1066 фотографий
Размер класса 4 составляет 1003 фотографий
Общий размер базы для обучения: 6688


In [ ]:
!rm -rf Test

In [ ]:
# Перенос файлов для теста в отдельное дерево папок, расчет размеров наборов данных

os.mkdir(TEST_PATH)                                        # Создание папки для тестовых данных
train_count = 0
test_count = 0

for class_name in CLASS_LIST:                              # Для всех классов по порядку номеров (их меток)
    class_path = f'{TRAIN_PATH}/{class_name}'              # Формирование полного пути к папке с изображениями класса
    test_path = f'{TEST_PATH}/{class_name}'                # Полный путь для тестовых данных класса
    class_files = os.listdir(class_path)                   # Получение списка имен файлов с изображениями текущего класса
    class_file_count = len(class_files)                    # Получение общего числа файлов класса
    os.mkdir(test_path)                                    # Создание подпапки класса для тестовых данных
    test_file_count = int(class_file_count * TEST_SPLIT)   # Определение числа тестовых файлов для класса
    test_files = class_files[-test_file_count:]            # Выделение файлов для теста от конца списка
    for f in test_files:                                   # Перемещение тестовых файлов в папку для теста
        os.rename(f'{class_path}/{f}', f'{test_path}/{f}')
    train_count += class_file_count                        # Увеличение общего счетчика файлов обучающего набора
    test_count += test_file_count                          # Увеличение общего счетчика файлов тестового набора

    print(f'Размер класса {class_name}: {class_file_count} машин, для теста выделено файлов: {test_file_count}')

print(f'Общий размер базы: {train_count}, выделено для обучения: {train_count - test_count}, для теста: {test_count}')

Размер класса 0: 1615 машин, для теста выделено файлов: 161
Размер класса 1: 1513 машин, для теста выделено файлов: 151
Размер класса 2: 1491 машин, для теста выделено файлов: 149
Размер класса 3: 1066 машин, для теста выделено файлов: 106
Размер класса 4: 1003 машин, для теста выделено файлов: 100
Общий размер базы: 6688, выделено для обучения: 6021, для теста: 667


In [ ]:
data_dir = '/content/Train'
batch_size = 16
image_size = (224,224)

In [ ]:
from tensorflow.keras.applications.resnet_v2 import preprocess_input

In [ ]:
def load_img(path):
    img = Image.open(path)
    #img = ImageOps.grayscale(img)
    #img = preprocess_input(img)
    img_np = np.array(img)
    img_np = img_np/255
    return img_np

In [ ]:
test_x = []
test_y = []
for i in os.listdir(TEST_PATH):
    for j in os.listdir(os.path.join(TEST_PATH,i)):
        path = os.path.join(TEST_PATH, str(i), j)
        img = load_img(path)
        img - np.array(img)
        test_y.append(int(i))
        test_x.append(img)

In [ ]:
train_x = []
train_y = []
for i in os.listdir(TRAIN_PATH):
    for j in os.listdir(os.path.join(TRAIN_PATH, i)):
        path = os.path.join(TRAIN_PATH, i, j)
        img = load_img(path)
        img - np.array(img)
        train_y.append(int(i))
        train_x.append(img)

In [ ]:
#train_x = preprocess_input(np.array(train_x))
#test_x = preprocess_input(np.array(test_x))
train_x = np.array(train_x)
test_x = np.array(test_x)
train_y = np.array(train_y)
test_y = np.array(test_y)
print(train_x.shape)
print(train_y.shape)
print(test_x.shape)
print(test_y.shape)

(6021, 224, 224, 3)
(6021,)
(667, 224, 224, 3)
(667,)


In [ ]:
import autokeras as ak
print(tf.config.list_physical_devices('GPU'))

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [ ]:
from keras.callbacks import ReduceLROnPlateau
reduceLROnPlateau = ReduceLROnPlateau(monitor='val_loss', factor=0.7, patience=3, verbose=1)

In [ ]:
clf = ak.ImageClassifier(num_classes=5,metrics='accuracy',overwrite=True, max_trials=5)

In [ ]:
clf.fit(
    x=train_x,
    y=train_y,
    # Use your own validation set.
    validation_split=0.15,
    epochs=20,
    callbacks =[reduceLROnPlateau]
)

Trial 1 Complete [00h 02m 40s]
val_loss: 6.953663955755474e-07

Best val_loss So Far: 6.953663955755474e-07
Total elapsed time: 00h 02m 40s

Search: Running Trial #2

Value             |Best Value So Far |Hyperparameter
resnet            |vanilla           |image_block_1/block_type
True              |True              |image_block_1/normalize
True              |False             |image_block_1/augment
True              |None              |image_block_1/image_augmentation_1/horizontal_flip
True              |None              |image_block_1/image_augmentation_1/vertical_flip
0                 |None              |image_block_1/image_augmentation_1/contrast_factor
0                 |None              |image_block_1/image_augmentation_1/rotation_factor
0.1               |None              |image_block_1/image_augmentation_1/translation_factor
0                 |None              |image_block_1/image_augmentation_1/zoom_factor
False             |None              |image_block_1/res_net_bloc

KeyboardInterrupt: 

In [ ]:
clf.evaluate(x=test_x,y=test_y)

NotFoundError: ./image_classifier/best_pipeline; No such file or directory